In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [7]:
df = pd.read_csv("../datasets/train_radiomics_hipocamp.csv")


X_test = pd.read_csv("../datasets/test_radiomics_hipocamp.csv")

In [8]:
df['Transition'] = df['Transition'].map({
    'CN-CN': 0,
    'CN-MCI': 1,
    'MCI-MCI': 2,
    'MCI-AD': 3,
    'AD-AD': 4
})
print(df['Transition'])

0      0
1      0
2      4
3      1
4      0
      ..
300    0
301    0
302    3
303    2
304    0
Name: Transition, Length: 305, dtype: int64


In [9]:
X = df.drop(['Transition'], axis=1)     
y = df['Transition'].to_frame()  

In [10]:
X_numerico = X.select_dtypes(include=[np.number])

X_test = X_test.select_dtypes(include=[np.number])

In [5]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
scaler = MinMaxScaler()
X_numerico = pd.DataFrame(scaler.fit_transform(X_numerico), columns=X_numerico.columns, index=X_numerico.index) #normalização

X_test = pd.DataFrame(scaler.fit_transform(X_test), columns=X_test.columns, index=X_test.index)

In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge, Lasso

# Lasso com validação cruzada
lasso_cv = GridSearchCV(Lasso(), param_grid={"alpha": [0.001, 0.01, 0.1, 1.0]}, cv=5)
lasso_cv.fit(X_numerico, y)
print("Melhor alpha para Lasso:", lasso_cv.best_params_)

print("Coeficientes Lasso:", lasso_cv.best_estimator_.coef_)

c:\Users\talex\miniconda3\envs\myenv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.526e-01, tolerance: 5.396e-02
  model = cd_fast.enet_coordinate_descent(
c:\Users\talex\miniconda3\envs\myenv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.499e+00, tolerance: 5.832e-02
  model = cd_fast.enet_coordinate_descent(
c:\Users\talex\miniconda3\envs\myenv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisati

Melhor alpha para Lasso: {'alpha': 0.01}
Coeficientes Lasso: [ 0.          0.          0.         ...  0.          0.17423733
 -1.52682274]


In [12]:
import pandas as pd
import numpy as np

# Obter os coeficientes do Lasso
lasso_coef = lasso_cv.best_estimator_.coef_

# Converter X_numerico em um DataFrame (se ainda não for)
if not isinstance(X_numerico, pd.DataFrame):
    X_numerico = pd.DataFrame(X_numerico)

# Selecionar as colunas cujos coeficientes são diferentes de zero
selected_columns = X_numerico.columns[np.abs(lasso_coef) > 0]

# Criar um novo dataset com as colunas selecionadas
X_reduzido = X_numerico[selected_columns]
X_test = X_test[selected_columns]

# Exibir o número de features selecionadas e as colunas escolhidas
print(f"Número de colunas selecionadas: {len(selected_columns)}")
print(f"Colunas selecionadas: {selected_columns.tolist()}")

# Agora você pode usar X_reduzido para testar outros modelos

Número de colunas selecionadas: 71
Colunas selecionadas: ['wavelet-LLH_glszm_SmallAreaHighGrayLevelEmphasis', 'wavelet-LLH_glszm_SmallAreaLowGrayLevelEmphasis', 'wavelet-LHL_glszm_SmallAreaEmphasis', 'wavelet-LHH_firstorder_Skewness', 'wavelet-LHH_glcm_Autocorrelation', 'wavelet-LHH_glcm_JointAverage', 'wavelet-LHH_gldm_DependenceNonUniformityNormalized', 'wavelet-LHH_ngtdm_Contrast', 'wavelet-HLL_firstorder_Maximum', 'wavelet-HLL_glcm_MCC', 'wavelet-HLL_gldm_LargeDependenceEmphasis', 'wavelet-HLL_glszm_SizeZoneNonUniformityNormalized', 'wavelet-HLH_glcm_Imc2', 'wavelet-HLH_glszm_LargeAreaLowGrayLevelEmphasis', 'wavelet-HLH_glszm_ZoneVariance', 'wavelet-HHL_glszm_SmallAreaEmphasis', 'wavelet-HHH_glcm_JointEntropy', 'wavelet-HHH_glcm_SumSquares', 'wavelet-HHH_gldm_HighGrayLevelEmphasis', 'wavelet-HHH_gldm_LowGrayLevelEmphasis', 'wavelet-HHH_glszm_LargeAreaHighGrayLevelEmphasis', 'wavelet-HHH_glszm_ZoneEntropy', 'wavelet-LLL_glszm_GrayLevelVariance', 'log-sigma-1-0-mm-3D_firstorder_Skewn

In [16]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import uniform, randint

# ================= Random Forest =================
rf = RandomForestClassifier(random_state=2024)

# Espaço de hiperparâmetros para RandomSearchCV
rf_param_dist = {
    'n_estimators': randint(100, 1000),
    'max_depth': randint(3, 20),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 20),
    'criterion': ['gini', 'entropy']
}

# RandomizedSearchCV para Random Forest
rf_random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=rf_param_dist,
    n_iter=50,
    scoring='accuracy',
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

rf_random_search.fit(X_reduzido, y)
print("Best parameters for Random Forest:", rf_random_search.best_params_)

# Avaliação no conjunto de teste
rf_best_model = rf_random_search.best_estimator_
y_pred_rf = rf_best_model.predict(X_test)
y_pred_rf

Fitting 5 folds for each of 50 candidates, totalling 250 fits


c:\Users\talex\miniconda3\envs\myenv\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Best parameters for Random Forest: {'criterion': 'entropy', 'max_depth': 16, 'min_samples_leaf': 18, 'min_samples_split': 10, 'n_estimators': 445}


array([0, 0, 2, 0, 0, 0, 4, 0, 3, 2, 0, 2, 0, 0, 3, 3, 0, 0, 0, 4, 3, 2,
       0, 3, 4, 0, 4, 4, 0, 0, 0, 0, 4, 4, 0, 3, 4, 0, 0, 0, 0, 0, 4, 0,
       0, 3, 0, 0, 3, 0, 0, 0, 3, 4, 0, 4, 0, 0, 4, 0, 0, 4, 0, 3, 0, 0,
       0, 0, 3, 0, 0, 3, 0, 3, 4, 3, 3, 0, 0, 0, 0, 3, 4, 0, 4, 0, 4, 2,
       0, 0, 4, 0, 4, 0, 0, 0, 4, 0, 0, 4], dtype=int64)

In [17]:
predictions = y_pred_rf
predictions = [round(pred) for pred in predictions]

reverse_mapping = {
    0: 'CN-CN',
    1: 'CN-MCI',
    2: 'MCI-MCI',
    3: 'MCI-AD',
    4: 'AD-AD'
}



# Garantir que as previsões são mapeadas para o dicionário
predictions = [reverse_mapping.get(int(round(pred)), 'Unknown') for pred in predictions]
predictions

['CN-CN',
 'CN-CN',
 'MCI-MCI',
 'CN-CN',
 'CN-CN',
 'CN-CN',
 'AD-AD',
 'CN-CN',
 'MCI-AD',
 'MCI-MCI',
 'CN-CN',
 'MCI-MCI',
 'CN-CN',
 'CN-CN',
 'MCI-AD',
 'MCI-AD',
 'CN-CN',
 'CN-CN',
 'CN-CN',
 'AD-AD',
 'MCI-AD',
 'MCI-MCI',
 'CN-CN',
 'MCI-AD',
 'AD-AD',
 'CN-CN',
 'AD-AD',
 'AD-AD',
 'CN-CN',
 'CN-CN',
 'CN-CN',
 'CN-CN',
 'AD-AD',
 'AD-AD',
 'CN-CN',
 'MCI-AD',
 'AD-AD',
 'CN-CN',
 'CN-CN',
 'CN-CN',
 'CN-CN',
 'CN-CN',
 'AD-AD',
 'CN-CN',
 'CN-CN',
 'MCI-AD',
 'CN-CN',
 'CN-CN',
 'MCI-AD',
 'CN-CN',
 'CN-CN',
 'CN-CN',
 'MCI-AD',
 'AD-AD',
 'CN-CN',
 'AD-AD',
 'CN-CN',
 'CN-CN',
 'AD-AD',
 'CN-CN',
 'CN-CN',
 'AD-AD',
 'CN-CN',
 'MCI-AD',
 'CN-CN',
 'CN-CN',
 'CN-CN',
 'CN-CN',
 'MCI-AD',
 'CN-CN',
 'CN-CN',
 'MCI-AD',
 'CN-CN',
 'MCI-AD',
 'AD-AD',
 'MCI-AD',
 'MCI-AD',
 'CN-CN',
 'CN-CN',
 'CN-CN',
 'CN-CN',
 'MCI-AD',
 'AD-AD',
 'CN-CN',
 'AD-AD',
 'CN-CN',
 'AD-AD',
 'MCI-MCI',
 'CN-CN',
 'CN-CN',
 'AD-AD',
 'CN-CN',
 'AD-AD',
 'CN-CN',
 'CN-CN',
 'CN-CN',
 'AD-AD',
 'CN

In [19]:
from xgboost import XGBClassifier

# ================= XGBoost =================
xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)

# Espaço de hiperparâmetros para RandomSearchCV
xgb_param_dist = {
    'n_estimators': randint(100, 1000),
    'learning_rate': uniform(0.01, 0.3),
    'max_depth': randint(3, 20),
    'subsample': uniform(0.5, 0.5),
    'colsample_bytree': uniform(0.5, 0.5),
    'gamma': uniform(0, 5)
}

# RandomizedSearchCV para XGBoost
xgb_random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=xgb_param_dist,
    n_iter=50,
    scoring='accuracy',
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

xgb_random_search.fit(X_reduzido, y)
print("Best parameters for XGBoost:", xgb_random_search.best_params_)

# Avaliação no conjunto de teste
xgb_best_model = xgb_random_search.best_estimator_
y_pred_xgb = xgb_best_model.predict(X_test)
y_pred_xgb

Fitting 5 folds for each of 50 candidates, totalling 250 fits


c:\Users\talex\miniconda3\envs\myenv\Lib\site-packages\xgboost\core.py:158: UserWarning: [22:22:47] WARNING: D:\bld\xgboost-split_1733179535861\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Best parameters for XGBoost: {'colsample_bytree': 0.9847923138822793, 'gamma': 3.8756641168055728, 'learning_rate': 0.29184968246925674, 'max_depth': 6, 'n_estimators': 369, 'subsample': 0.8636359979282104}


array([2, 0, 2, 2, 0, 0, 4, 0, 3, 3, 0, 2, 0, 0, 4, 2, 0, 0, 2, 2, 2, 2,
       0, 3, 3, 2, 4, 4, 0, 0, 0, 2, 3, 4, 0, 2, 4, 2, 0, 0, 3, 0, 4, 0,
       0, 4, 2, 0, 3, 0, 2, 0, 3, 4, 0, 4, 0, 2, 4, 2, 0, 4, 0, 3, 0, 0,
       2, 0, 3, 0, 0, 3, 0, 2, 3, 3, 3, 0, 0, 0, 0, 2, 3, 0, 3, 3, 4, 2,
       2, 2, 3, 0, 3, 0, 0, 0, 4, 0, 0, 3], dtype=int64)

In [20]:
predictions = y_pred_xgb
predictions = [round(pred) for pred in predictions]

reverse_mapping = {
    0: 'CN-CN',
    1: 'CN-MCI',
    2: 'MCI-MCI',
    3: 'MCI-AD',
    4: 'AD-AD'
}



# Garantir que as previsões são mapeadas para o dicionário
predictions = [reverse_mapping.get(int(round(pred)), 'Unknown') for pred in predictions]

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import numpy as np

# ================= Logistic Regression =================
lr = LogisticRegression(multi_class='multinomial', solver='saga', max_iter=5000, random_state=42)

# Espaço de hiperparâmetros para GridSearchCV
lr_param_grid = {
    'penalty': ['l1', 'l2', 'elasticnet'],  # Removido 'none' para compatibilidade com 'saga'
    'C': np.logspace(-4, 4, 20),  # Valores de regularização
    'l1_ratio': [0.1, 0.5, 0.9]  # Apenas relevante para penalty='elasticnet'
}

# GridSearchCV para Regressão Logística
lr_grid_search = GridSearchCV(
    estimator=lr,
    param_grid=lr_param_grid,
    scoring='f1_weighted',  # Corrigido 'f1_weigthed' -> 'f1_weighted'
    cv=5,
    verbose=2,
    n_jobs=-1
)

# Ajustar GridSearchCV no conjunto de dados
lr_grid_search.fit(X_reduzido, y)

# Melhor conjunto de hiperparâmetros
print("Best parameters for Logistic Regression:", lr_grid_search.best_params_)

# Avaliação no conjunto de teste
lr_best_model = lr_grid_search.best_estimator_
y_pred_lr = lr_best_model.predict(X_test)
y_pred_lr


Fitting 5 folds for each of 180 candidates, totalling 900 fits


c:\Users\talex\miniconda3\envs\myenv\Lib\site-packages\sklearn\linear_model\_logistic.py:1197: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
c:\Users\talex\miniconda3\envs\myenv\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\talex\miniconda3\envs\myenv\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Best parameters for Logistic Regression: {'C': 11.288378916846883, 'l1_ratio': 0.1, 'penalty': 'l2'}


array([3, 2, 2, 0, 0, 3, 3, 0, 3, 2, 0, 0, 3, 3, 4, 0, 0, 3, 2, 2, 0, 2,
       0, 0, 4, 0, 4, 4, 1, 2, 0, 2, 4, 4, 2, 3, 4, 2, 2, 2, 2, 0, 3, 2,
       1, 2, 2, 0, 3, 3, 2, 0, 2, 3, 0, 3, 0, 2, 4, 0, 0, 4, 0, 3, 3, 0,
       2, 2, 4, 2, 0, 3, 0, 3, 4, 2, 2, 3, 2, 0, 0, 0, 4, 0, 4, 1, 4, 3,
       2, 0, 3, 2, 3, 3, 1, 2, 4, 3, 0, 3], dtype=int64)

In [30]:
predictions = y_pred_lr
predictions = [round(pred) for pred in predictions]

reverse_mapping = {
    0: 'CN-CN',
    1: 'CN-MCI',
    2: 'MCI-MCI',
    3: 'MCI-AD',
    4: 'AD-AD'
}



# Garantir que as previsões são mapeadas para o dicionário
predictions = [reverse_mapping.get(int(round(pred)), 'Unknown') for pred in predictions]

In [10]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# Definir o modelo SVM
model = SVC(random_state=2022)

y = np.ravel(y)

# Definir a grade de parâmetros
param_grid = {
    'C': [0.1, 1, 10, 100],  # Regularização
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],  # Tipos de kernel
    'degree': [2, 3, 4],  # Apenas relevante para o kernel 'poly'
    'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1],  # Kernel 'rbf', 'poly', 'sigmoid'
    'coef0': [0.0, 0.1, 0.5, 1.0]  # Apenas para kernels 'poly' e 'sigmoid'
}

# Configurar o GridSearchCV
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='f1_weighted',  # Métrica de avaliação
    cv=5,  # Número de folds para validação cruzada
    verbose=3,  # Para acompanhar o progresso
    n_jobs=-1  # Usar todos os núcleos disponíveis
)

# Realizar o treinamento
grid_search.fit(X_reduzido, y)

# Melhor estimador e resultados
print("Melhores parâmetros:", grid_search.best_params_)
print("Melhor pontuação:", grid_search.best_score_)

# Obter o melhor modelo e realizar treinamento final
model = grid_search.best_estimator_
model.fit(X_reduzido, y)


Fitting 5 folds for each of 1152 candidates, totalling 5760 fits
Melhores parâmetros: {'C': 100, 'coef0': 1.0, 'degree': 2, 'gamma': 'auto', 'kernel': 'sigmoid'}
Melhor pontuação: 0.49741789243626605


SVC(C=100, coef0=1.0, degree=2, gamma='auto', kernel='sigmoid',
    random_state=2022)

In [11]:
predictions = model.predict(X_test)
predictions

array([3, 2, 2, 0, 0, 4, 4, 0, 3, 2, 0, 3, 3, 0, 4, 2, 0, 3, 2, 4, 2, 0,
       2, 0, 4, 2, 4, 4, 0, 2, 0, 2, 4, 4, 2, 3, 4, 0, 2, 2, 2, 0, 4, 0,
       0, 4, 2, 0, 4, 2, 2, 2, 4, 4, 0, 4, 0, 0, 4, 0, 0, 4, 0, 4, 3, 0,
       2, 2, 4, 0, 0, 3, 0, 4, 4, 2, 2, 2, 2, 0, 0, 0, 4, 0, 4, 0, 4, 3,
       2, 2, 3, 2, 3, 3, 0, 3, 4, 2, 0, 3], dtype=int64)

In [12]:
predictions = [round(pred) for pred in predictions]

reverse_mapping = {
    0: 'CN-CN',
    1: 'CN-MCI',
    2: 'MCI-MCI',
    3: 'MCI-AD',
    4: 'AD-AD'
}



# Garantir que as previsões são mapeadas para o dicionário
predictions = [reverse_mapping.get(int(round(pred)), 'Unknown') for pred in predictions]

In [7]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Instanciar o modelo KNN
knn = KNeighborsClassifier()

# Espaço de hiperparâmetros para o GridSearchCV
knn_param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],  # Testar diferentes números de vizinhos
    'weights': ['uniform', 'distance'],  # Pesos
    'metric': ['euclidean', 'manhattan', 'minkowski']  # Métrica de distância
}

# Configuração do GridSearchCV
knn_grid_search = GridSearchCV(
    estimator=knn,
    param_grid=knn_param_grid,
    scoring='f1_weighted',  # Ou outra métrica relevante (accuracy, recall, etc.)
    cv=5,  # Número de folds na validação cruzada
    verbose=2,
    n_jobs=-1  # Paralelizar para acelerar o treinamento
)

# Ajustar o modelo aos dados de treino
knn_grid_search.fit(X_numerico, y)

# Melhor conjunto de hiperparâmetros
print("Best parameters for KNN:", knn_grid_search.best_params_)


Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best parameters for KNN: {'metric': 'manhattan', 'n_neighbors': 11, 'weights': 'distance'}


c:\Users\talex\miniconda3\envs\myenv\Lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [8]:
# Melhor modelo encontrado no GridSearch
knn_best_model = knn_grid_search.best_estimator_

# Previsões no conjunto de teste
y_pred_knn = knn_best_model.predict(X_test)

In [9]:
predictions = y_pred_knn
predictions = [round(pred) for pred in predictions]

reverse_mapping = {
    0: 'CN-CN',
    1: 'CN-MCI',
    2: 'MCI-MCI',
    3: 'MCI-AD',
    4: 'AD-AD'
}



# Garantir que as previsões são mapeadas para o dicionário
predictions = [reverse_mapping.get(int(round(pred)), 'Unknown') for pred in predictions]

In [10]:
data = pd.DataFrame({
    'RowId': np.arange(1, len(predictions) + 1), 
    'Result': predictions
})

data.to_csv('KNN.csv', index=False)